In [1]:
import sys
import numpy as np
from scipy.sparse import csr_matrix as csr_mat
import time
from multiprocessing import Pool    
from quspin.tools.lanczos import lanczos_full, expm_lanczos
import numba as nb


In [2]:
# auxiliary functions to create basis
def first_state(L,up=False):
        if up:
            n_upspins = L//2+1
        else:
            n_upspins = L//2
        return (1 << n_upspins) - 1
def next_state(state):
        t = (state | (state - 1)) + 1
        return t | ((((t & -t) // (state & -state)) >> 1) - 1)
def last_state(L,up=False):
        if up:
            n_upspins = L//2+1
        else:
            n_upspins = L//2
        return ((1 << n_upspins) - 1) << (L - n_upspins)

#get number of spin states Ns if hole is present  

def N_spinStates(L,hole_present=True):
    if L%2==1:
        raise ValueError('Only even number of sites possible')
    if hole_present:
        N=np.math.factorial(L)/(2*(np.math.factorial(L/2))**2)
    else:
        N=np.math.factorial(L)/((np.math.factorial(L/2))**2)
    return int(N)

# create spin basis for Sz=0
def Sz0basis(L,up=False):
    basis = []
    state = first_state(L,up)
    end_state = last_state(L,up)
    while state <= end_state:
        basis.append(state)
        state = next_state(state)
    return np.array(basis)

# create basis for t-J model
def tJ_basis_down(L):
    basis=np.array([])
    for j in range(0,L):
        basis=np.append(basis,Sz0basis(L-1,up=False)+j*2**(L-1))
    return basis.astype('int32')


#get value of state "state" on site "site"
@nb.jit(nopython=True, cache=True)
def get_site_value(state, site):
    return (state >> site) & 1

@nb.jit(nopython=True, cache=True)
def reducebasis(spin_basis, L, hole_pos):
    """reduces basis by hole up"""
    reduced_basis=[]
    for state in spin_basis:
        if get_site_value(state, hole_pos)==1:
            reduced_basis.append(state)
    return reduced_basis

@nb.jit(nopython=True, cache=True)
def get_hole_pos(state,L):
    return int(state/(2**(L-1)))

def csr_matrix(rows,cols, data, s):
    # takes rows, cols, data as a python list
    return csr_mat((data, (rows, cols)), shape=(s,s))

@nb.jit(nopython=True, cache=True)
def list_add(list1,a):
    return [x+a for x in list1]

# create heisenberg chain hamiltonian in csr format
@nb.jit(nopython=True, cache=True)
def make_H_spin(L, J, E, spin_basis, hole_pos=None, hole_present=True):

    # bonds with periodic bc

    heisenberg_bonds = [(site, (site+1)%L) for site in range(L)]

    h_rows = []
    h_cols = []
    h_data = []

    # run through spin state basis
    for state_index, state in enumerate(spin_basis):

        # diagonal S_z interaction
        diagonal = 0

        if hole_present:
            for bond in heisenberg_bonds:
                if bond[0]!=hole_pos and bond[1]!=hole_pos:
                    if get_site_value(state, bond[0]) == get_site_value(state, bond[1]):
                        diagonal += J/4
                    else:
                        diagonal -= J/4
        else:
            for bond in heisenberg_bonds:
                if get_site_value(state, bond[0]) == get_site_value(state, bond[1]):
                    diagonal += J/4
                else:
                    diagonal -= J/4
                
        
                
        #constant terms depending if hole is present 
        
        #Wohlfeld
        #"""
        if hole_present:
            diagonal += (L-1)*E-(L-2)*J/4   #for the constant -J_ii1/4 n_i n_i+1 and E_ii1 ni term
        else:                               #in the Wohlfeld definition E has to be negative
            diagonal += L*E-L*J/4
        """
        #Schlappa
        if hole_present:
            diagonal += -E                  #has to be postive
        """

        h_rows.append(state_index)
        h_cols.append(state_index)
        h_data.append(diagonal)

        # offdiagonal S_x and S_y interaction
        
        if hole_present:
            for bond in heisenberg_bonds:
                if bond[0]!=hole_pos and bond[1]!=hole_pos:
                    flipmask = (1 << bond[0]) | (1 << bond[1])
                    if get_site_value(state, bond[0]) != get_site_value(state, bond[1]):
                        new_state = state ^ flipmask
                        try:
                            new_state_index = spin_basis.index(new_state)
                            h_rows.append(state_index)
                            h_cols.append(new_state_index)
                            h_data.append(J/2)
                        except:
                            pass
        else:
            for bond in heisenberg_bonds:
                flipmask = (1 << bond[0]) | (1 << bond[1])
                if get_site_value(state, bond[0]) != get_site_value(state, bond[1]):
                    new_state = state ^ flipmask
                    new_state_index = spin_basis.index(new_state)
                    h_rows.append(state_index)
                    h_cols.append(new_state_index)
                    h_data.append(J/2)

    return h_rows, h_cols, h_data

        


In [3]:
# Gram Schmidt orthogonalization
def gram_schmidt(Q):
    n = Q.shape[0] 
    for j in range(n):
        Q_im = Q[0, :]*0
        for k in range(j):
            Q_im += np.dot(Q[k, :], Q[j, :])/np.linalg.norm(Q[k, :]) * Q[k, :] #v=v-(w,v)*w 
        Q[j, :] -= Q_im
    return Q

# Lanczos algorithm
def lanczos(H, eig_max=50, num_it=100, precision=1e-8, max_iterations=1000, gs=True, printConv=False, use_prec_val=False):
    
    H_size = H.get_shape()[0]
    alphas = []
    betas = [0.]
    t_eigval_all=[[]]
    v0 = np.zeros(H_size, dtype=float)
    v1 = np.random.rand(H_size)
    v1 /= np.linalg.norm(v1)
    w = np.zeros(H_size, dtype=float)
    Q=np.array([v1])
    alpha = 0.
    beta = 0.
    prev_energy = 0
    
    for iteration in range(1, max_iterations):
        
        #Lanczos iteration step
        w = H.dot(v1)
        alpha = np.real(np.vdot(v1,w))     #/np.vdot(v1,v1))          #alpha=<v1|H|v1>
        w = w - alpha*v1 - beta*v0             
        v0 = np.copy(v1)                                   #v1 of last iteration is v0 of current iteration
        beta = np.real(np.sqrt(np.vdot(w,w)))              #beta=<v1|H^\dagger H|v1>-alpha^2-beta^2
        v1 = 1/beta*w                                      #beta |v1>=(H-alpha)|v1>- beta |v0>
        alphas.append(alpha)
        betas.append(beta)

        
        #build T-matrix
        t_matrix = np.diag(np.array(alphas)) + \
                   np.diag(np.array(betas)[1:-1],k=1) + \
                   np.diag(np.array(betas)[1:-1],k=-1)

        t_eigval, t_eigvec = np.linalg.eigh(t_matrix) #The column v[:, i] is the normalized eigenvector corresponding to the eigenvalue w[i]
        GS_eigvec = t_eigvec[:,t_eigval.argmin()]
        if iteration < eig_max:
            t_eigval_all.append([])
            
        for eig in range(eig_max):
            if iteration > eig:
                t_eigval_all[eig].append(t_eigval[eig])
        
        #return if converged
        if iteration==num_it or (np.abs(min(t_eigval) - prev_energy) < precision and use_prec_val):
            #print("Lanczos converged in", iteration, "steps")
            if printConv:
                fig, ax = plt.subplots(figsize=(14,14))
                for eig in range(eig_max):
                    ax.plot(range(eig, iteration), t_eigval_all[eig], 'b-')
                ax.set_xlabel('Iterations')
                ax.set_ylabel('first '+str(eig_max)+' eigenvalues')
                fig.savefig('convergence_eigval.pdf')
                
                fig = plt.figure(figsize=(12,6))
                sub=plt.subplot(1,2,1)
                sub.plot(range(0, iteration), alphas, 'b-', label='alpha')
                sub=plt.subplot(1,2,2)
                sub.plot(range(0, iteration+1), betas, 'r-', label='beta')
                #ax.legend()
            return t_eigval, t_eigvec, Q#, np.array(alphas), np.array(betas[1:])
        prev_energy = min(t_eigval)
        Q=np.append(Q,np.array([v1]),axis=0)
        
        # Gram Schmitt
        if gs:
            Q=gram_schmidt(Q)
            v1=Q[-1,:]  


In [4]:
"""trail parameters"""
L       = 4
Ns      = N_spinStates(L)
J       = 0.24
t       = -0.08
E       = -2

filename_log="LOG_spin.txt"
filelog=open(filename_log,'w')
filelog.close()

In [5]:
"""approximative Calculations"""
start = time.time()

"""spin only calculations"""
spin_basis=list(Sz0basis(L).astype('int32'))
typed_spin_basis = nb.typed.List()
[typed_spin_basis.append(x) for x in spin_basis]

h_rows, h_cols, h_data = make_H_spin(L, J, E, typed_spin_basis, hole_present=False)
H_spin=csr_matrix(h_rows, h_cols, h_data, N_spinStates(L, hole_present=False))
H_spin_eval, t_spin_evec, Q_spin = lanczos(H_spin, use_prec_val=True)
H_spin_evec=np.dot(np.transpose(Q_spin), t_spin_evec)

#gs energy and vector of spin only Hamiltonian
E_spin_GS=H_spin_eval[0]
phi=H_spin_evec[:,H_spin_eval.argmin()]

np.save('phi_'+str(L)+'.npy',phi)

end = time.time()
filelog=open(filename_log,'a')
data_stringlog = str('CPU time of the spin groundstate search')+'     '+str(np.round(end - start,3))+'\n'
filelog.write(data_stringlog)
filelog.close()
